In [1]:
import os
import json
import psycopg2

/home/devopsgong/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
!ls

2017-11-10.json  batch_size.config  json2db.ipynb  README.md  test1.json


In [3]:
# config params
file_in = 'test1.json'

db_host = os.environ.get('AWS_PG_DB_HOST')
db_name = os.environ.get('AWS_PG_DB_NAME')
db_user = os.environ.get('AWS_PG_DB_USER')
password = os.environ.get('AWS_PG_DB_PASS')
schema_name = 'public'

# table to Json obj
mapTable2JsonObj = {'dm_orders':'orders', 'dm_line_items':'line_items'}

In [7]:
with open(file_in) as f:
    s = f.read()

In [8]:
dic = json.loads(s)

In [9]:
dic

{'orders': [{'id': 11748933635,
   'email': 'censored@censored.com',
   'closed_at': None,
   'created_at': '2017-10-30T19:58:29-04:00',
   'updated_at': '2017-10-30T19:59:30-04:00',
   'number': 97171,
   'note': None,
   'token': 'censored',
   'gateway': 'cash',
   'test': False,
   'total_price': '130.64999999999998',
   'subtotal_price': '120.0',
   'total_weight': 6,
   'total_tax': '10.649999999999999',
   'taxes_included': False,
   'currency': 'USD',
   'financial_status': 'paid',
   'confirmed': True,
   'total_discounts': '0.00',
   'total_line_items_price': '120.0',
   'cart_token': 'censored',
   'buyer_accepts_marketing': False,
   'name': 'GLO37-2010000000',
   'referring_site': None,
   'landing_site': None,
   'cancelled_at': None,
   'cancel_reason': None,
   'total_price_usd': '130.64999999999998',
   'checkout_token': 'censored',
   'reference': '9925E904-4A8D-4520-9F2F-46B847BDE46D',
   'user_id': 110585347,
   'location_id': 371291,
   'source_identifier': '371291

In [10]:
len(dic[mapTable2JsonObj['dm_orders']])

2

In [11]:
i_orders = 0  
# get order fields
order_id = dic[mapTable2JsonObj['dm_orders']][i_orders]['id']

In [13]:
# get line_itesm
dic[mapTable2JsonObj['dm_orders']][i_orders][mapTable2JsonObj['dm_line_items']]

[{'id': 23495245827,
  'variant_id': 36499702851,
  'quantity': 1,
  'product_id': 9096535107}]

In [14]:
# connect to PostgreSQL
db_connection_string = f"dbname='{db_name}' user='{db_user}' host='{db_host}' password='{password}'"
connection = psycopg2.connect(db_connection_string)
cur = connection.cursor()

work on table - `dm_line_items`

In [15]:

tbl_name = 'dm_line_items'

In [16]:
sql_coldef = f"""
    SELECT sc.table_name, sc.column_name, sc.data_type
    FROM information_schema.columns sc
    WHERE table_schema = '{schema_name}'
      AND table_name = '{tbl_name}'
      order by table_name,ordinal_position;
"""

In [17]:
cur.execute(sql_coldef)
col_defs = cur.fetchall()

In [18]:
col_defs

[('dm_line_items', 'id', 'bigint'),
 ('dm_line_items', 'order_id', 'bigint'),
 ('dm_line_items', 'variant_id', 'bigint'),
 ('dm_line_items', 'quantity', 'integer'),
 ('dm_line_items', 'product_id', 'bigint')]

In [19]:
# get column info
col_list = [(col[1],col[2]) for col in col_defs]

col_list

[('id', 'bigint'),
 ('order_id', 'bigint'),
 ('variant_id', 'bigint'),
 ('quantity', 'integer'),
 ('product_id', 'bigint')]

In [20]:
# build SQL insert
col_list_str = ",".join([f"\"{c[0]}\""  for c in col_list])

In [22]:
i_line_items = 0
val_list_str = ""
iv = 0
for c in col_list:
    col_name, col_type = c[0], c[1]
    if col_name == 'order_id':
        val = order_id
    else:
        val = dic[mapTable2JsonObj['dm_orders']][i_orders][mapTable2JsonObj['dm_line_items']][i_line_items][col_name]
    if val is None:
        val = "NULL"
    else:
        if "character" in col_type or "text" in col_type or "timestamp" in col_type:
            val = f"'{val}'"
        elif "boolean" in col_type:
            if val:
                val = "CAST( 1 AS BOOLEAN )"
            else:
                val = "CAST( 0 AS BOOLEAN )"
        else:
            val = f"{val}"

    if iv:
        val_list_str += ',' + val
    else:
        val_list_str += val
        
    iv += 1

In [23]:
sql_insert = f"""
    INSERT INTO \"{tbl_name}\" ({col_list_str})
    VALUES ({val_list_str});
""" 
sql_insert

'\n    INSERT INTO "dm_line_items" ("id","order_id","variant_id","quantity","product_id")\n    VALUES (23495245827,11748933635,36499702851,1,9096535107);\n'

In [24]:
cur.execute(sql_insert)

connection.commit()  # write to db

In [26]:
# build SQL select
col_list_str = ",".join([f"\"{c[0]}\""  for c in col_list])
sql_select = f"""
    SELECT {col_list_str} FROM \"{tbl_name}\";
""" 
sql_select

'\n    SELECT "id","order_id","variant_id","quantity","product_id" FROM "dm_line_items";\n'

In [27]:
cur.execute(sql_select)
rows = cur.fetchall()

In [28]:
rows

[(23495245827, 11748933635, 36499702851, 1, 9096535107),
 (23495245827, 11748933635, 36499702851, 1, 9096535107)]

In [29]:
# cleanup
sql_delete = f"""
    DELETE FROM \"{tbl_name}\";
""" 
sql_delete

cur.execute(sql_delete)
connection.commit()

In [30]:
!pwd

/home/devopsgong/projects/insight_ds/coding_challenges/data_migration/input


In [ ]:
connection.close()